In [1]:
import pandas as pd
import subprocess

In [2]:
lines = []
with open("openTenderNotice-ouvertAvisAppelOffres.csv", "r") as f:
    for l in f:
        lines.append(l)

In [3]:
class PreviousLine:
    def __init__(self, a, b, c, d):
        self.inQuotes = a
        self.s = b
        self.strings = c
        self.i = d
        
    def __str__(self):
        return str((self.inQuotes, self.s, "..." + self.strings[-1], self.i))


class CSVData:
    def __init__(self, lines):
        self.cols = [w.split('"')[1] for w in lines[0].split(',')]
        self.nCols = len(self.cols)
        self.lines = lines
        self.currentLineIndex = 1
        #
        cols = {w:i for i, w in enumerate(self.cols) if not "-fra" in w}
        self.wantedColumns = {(w.split('-')[0]): i for w, i in cols.items()}
        self.nWantedColumns = len(self.wantedColumns)
            
    def getNextRecords(self, k):
        records = []
        for i in range(k):
            try:
                record, self.currentLineIndex = self.nextRecord(self.currentLineIndex)
            except Exception as e:
                print((i, e))
                return records
            records.append(record)
        return records
        
    def nextRecord(self, lineIndex):
        i = lineIndex
        strings, previousLine = self.stringsFromLine(self.lines[i], None)
        #print(previousLine)
        
        while previousLine:
            i += 1
            strings, previousLine = self.stringsFromLine(self.lines[i], previousLine)
            #print(previousLine)   
        return strings, i + 1

    def stringsFromLine(self, l, previousLine=None):
        if previousLine:
            inQuotes = previousLine.inQuotes
            s = previousLine.s
            strings = previousLine.strings
            i = previousLine.i
        else:
            inQuotes = False
            s = ''
            strings = []
            i = 0
        for c in l:
            if c == '"':
                if inQuotes:
                    inQuotes = False
                    if i == self.nCols - 1:
                        strings.append(s)
                        i += 1
                else:
                    s = ''
                    inQuotes = True
            else:
                if inQuotes:
                    if c == "\n":
                        s += "\\n"
                    else:
                        s += c
                elif c == ',':
                    strings.append(s)
                    i += 1
        if i < self.nCols:
            # got not all columns
            previousLine = PreviousLine(inQuotes, s, strings, i)
            return (strings, previousLine)
        else:
            return (strings, None)

    def extractFields(self, record, fields=None):
        if not fields:
            fields = self.wantedColumns 
        return [r for i, r in enumerate(record) if i in fields.values()]

    def toDataFrame(self, records, fields=None):
        if not fields:
           fields = self.wantedColumns 
        recordsDict = {i: self.extractFields(record, fields) for i, record in enumerate(records)}
        df = pd.DataFrame.from_dict(recordsDict, orient='index', columns=fields.keys())
        return df
    


In [4]:
csvData = CSVData(lines)

In [5]:
importantColumns = ['title', 'tenderClosingDate', 'tenderStatus', 'unspscDescription', 'contractingEntityName', 'tenderDescription', 'noticeURL', 'attachment']

In [173]:
records = []
records.extend(csvData.getNextRecords(1000))

(684, IndexError('list index out of range'))


In [6]:
csvData = CSVData(lines)
records = []
records.extend(csvData.getNextRecords(684))

In [176]:
len(records)

681

In [7]:
r = csvData.getNextRecords(1)

(0, IndexError('list index out of range'))


In [8]:
df = csvData.toDataFrame(records)
df_ = df.loc[:, importantColumns]

In [20]:
df.to_pickle("df.pkl")
df.to_csv("df.csv")

In [9]:
df_.head()

,title,tenderClosingDate,tenderStatus,unspscDescription,contractingEntityName,tenderDescription,noticeURL,attachment
0,Electrical and Electrical Products (New Brunsw...,2024-06-25T11:00:00,Open,*Electrical components,Department of National Defence,The Department of National Defence has a requi...,,https://canadabuys.canada.ca/sites/default/fil...
1,"Dental Services for Manitoba Region, FNIHB Pro...",2025-03-31T14:00:00,Open,*Dental services,Indigenous Services Canada,"Indigenous Services Canada (ISC), First Nation...",,https://canadabuys.canada.ca/sites/default/fil...
2,Water and Wastewater Audits - Alberta / Britis...,2024-07-16T13:00:00,Open,*Environmental auditing,Melissa Smith,DESCRIPTION:\nAgriculture and Agri-Food Canada...,,https://canadabuys.canada.ca/sites/default/fil...
3,Request for Information-Indigenous Businesses,2024-08-30T14:00:00,Open,*Information technology consultation services\...,Employment and Social Development Canada,Employment and Social Development Canada (ESDC...,,https://canadabuys.canada.ca/sites/default/fil...
4,One (1) Intermediate Technical Writer,2024-07-05T14:00:00,Open,*Technical writing,Department of National Defence,NOTICE OF PROPOSED PROCUREMENT\n\nFile Number:...,,


In [ ]:
minimalColumns = ['title', 'tenderClosingDate', 'unspscDescription', 'contractingEntityName', 'tenderDescription']
df_ = df.loc[:, minimalColumns]

In [188]:
df_.tenderStatus.value_counts()

tenderStatus
Open    561
,        62
         61
Name: count, dtype: int64

In [189]:
df_.noticeURL.value_counts()

noticeURL
                                                                                     252
,                                                                                    114
https://discovery.ariba.com/rfx/19456899?extsite=MTk0NTY4OTlfQU4wMTQ4NDA1MDk5Mg==      1
https://discovery.ariba.com/rfx/19894579?extsite=MTk4OTQ1NzlfQU4wMTQ4NDA1MDk5Mg==      1
https://discovery.ariba.com/rfx/19870581?extsite=MTk4NzA1ODFfQU4wMTQ4NDA1MDk5Mg==      1
                                                                                    ... 
https://www.merx.com/public/solicitations/2908823323/abstract?language=EN              1
https://discovery.ariba.com/rfx/19771075?extsite=MTk3NzEwNzVfQU4wMTQ4NDA1MDk5Mg==      1
https://discovery.ariba.com/rfx/19651439?extsite=MTk2NTE0MzlfQU4wMTQ4NDA1MDk5Mg==      1
https://discovery.ariba.com/rfx/19781390?extsite=MTk3ODEzOTBfQU4wMTQ4NDA1MDk5Mg==      1
https://discovery.ariba.com/rfx/19953509?extsite=MTk5NTM1MDlfQU4wMTQ4NDA1MDk5Mg==      1
Name: count

In [10]:
minimalColumns = ['title', 'tenderClosingDate', 'unspscDescription', 'contractingEntityName', 'tenderDescription']
df_ = df.loc[:, minimalColumns]
df_.head()

,title,tenderClosingDate,unspscDescription,contractingEntityName,tenderDescription
0,Electrical and Electrical Products (New Brunsw...,2024-06-25T11:00:00,*Electrical components,Department of National Defence,The Department of National Defence has a requi...
1,"Dental Services for Manitoba Region, FNIHB Pro...",2025-03-31T14:00:00,*Dental services,Indigenous Services Canada,"Indigenous Services Canada (ISC), First Nation..."
2,Water and Wastewater Audits - Alberta / Britis...,2024-07-16T13:00:00,*Environmental auditing,Melissa Smith,DESCRIPTION:\nAgriculture and Agri-Food Canada...
3,Request for Information-Indigenous Businesses,2024-08-30T14:00:00,*Information technology consultation services\...,Employment and Social Development Canada,Employment and Social Development Canada (ESDC...
4,One (1) Intermediate Technical Writer,2024-07-05T14:00:00,*Technical writing,Department of National Defence,NOTICE OF PROPOSED PROCUREMENT\n\nFile Number:...


In [193]:
list(df_.unspscDescription.value_counts().index)

['',
 ',',
 '*Product and material trailers',
 '*Laboratory and scientific equipment',
 '*Office furniture',
 '*Architectural engineering',
 '*Heavy construction machinery and equipment',
 '*Water transport vessel maintenance and repair services',
 '*Motor vehicles',
 '*Computer services',
 '*Building and Facility Construction and Maintenance Services',
 '*Commercial marine craft',
 '*Software',
 '*Nonresidential building construction services',
 '*Clothing',
 '*Cleaning and janitorial services',
 '*Audit services',
 '*Decontamination services',
 '*Information technology consultation services',
 '*Building and facility maintenance and repair services',
 '*Laboratory and Measuring and Observing and Testing Equipment',
 '*Building and Facility Construction and Maintenance Services\\n*Nonresidential building construction services',
 '*Material handling machinery and equipment',
 '*Professional engineering services',
 '*Snow plow truck',
 '*Product research and development service',
 '*Roo

In [14]:
specificUnspscDescriptions = \
    [    
        '*Software',
        '*Computer services',
        '*Information technology consultation services',
        '*Product research and development service',
        '*Media monitoring service',
        '*Information Technology Service Delivery',
        '*Engineering and Research and Technology Based Services',
        '*Product testing',
        '*Editing services',
        '*Information retrieval systems',
        '*Writing and translations',
        '*Technical writing',
        '*Analytical or scientific software',
        '*Industrial optics'
    ]

In [12]:
f = lambda x: '*Technical writing' in x
df2 = df_.loc[df_.unspscDescription.apply(f), :]
df2

,title,tenderClosingDate,unspscDescription,contractingEntityName,tenderDescription
4,One (1) Intermediate Technical Writer,2024-07-05T14:00:00,*Technical writing,Department of National Defence,NOTICE OF PROPOSED PROCUREMENT\n\nFile Number:...
393,Industry Sustainment Business Case Analysis (S...,2025-06-30T14:00:00,*Water quality assessment services\n*Water tes...,Procurement Innovation Initiatives Directorate...,Industry Sustainment Business Case Analysis (S...


In [13]:
f = lambda x: '*Analytical or scientific software' in x
df2 = df_.loc[df_.unspscDescription.apply(f), :]
df2

,title,tenderClosingDate,unspscDescription,contractingEntityName,tenderDescription
317,Invitation to Qualify to Artificial Intelligen...,2025-09-30T14:00:00,*Analytical or scientific software,Department of Public Works and Government Serv...,This publication replaces the following AI Sou...


In [201]:
f = lambda x: '*Media monitoring service' in x
df2 = df_.loc[df_.unspscDescription.apply(f), :]
df2

,title,tenderClosingDate,unspscDescription,contractingEntityName,tenderDescription
174,Reissuance - Media Monitoring Services - EN578...,2028-12-31T14:00:00,*Media monitoring service,Department of Public Works and Government Serv...,This event is to reissue on CanadaBuys the Req...
497,RFQ Canadian and International Ethnic Media Mo...,2024-07-03T14:00:00,*Media monitoring service,Department of Public Works and Government Serv...,RFQ under supply arrangement EN578-191713 for ...


In [202]:

f = lambda x: '*Information retrieval systems' in x
df2 = df_.loc[df_.unspscDescription.apply(f), :]
df2

,title,tenderClosingDate,unspscDescription,contractingEntityName,tenderDescription
248,E60PI-23AGNT-C - Ongoing Qualification to RFSA...,2029-02-09T14:00:00,*Information retrieval systems,Department of Public Works and Government Serv...,THIS IS AN ONGOING REQUEST FOR SUPPLY ARRANGEM...


In [15]:
dfs = []
for c in specificUnspscDescriptions:
    f = lambda x: c in x
    df2 = df_.loc[df_.unspscDescription.apply(f), :]
    dfs.append(df2)

In [16]:
df_out = pd.concat(dfs)[['title', 'tenderDescription']]
df_out.head(3)

,title,tenderDescription
267,Request for Information for Environment and Cl...,ECCC is launching this RFI to engage with and ...
279,CBSO-Application Portfolio Management (GC APM)...,AMENDMENT TO CLOSING TIME: Please disregard th...
283,Software Licensing Supply Arrangement (SLSA) RFSA,IMPORTANT NOTICE: Please submit offers to the ...


In [17]:
df_out.tail()

,title,tenderDescription
4,One (1) Intermediate Technical Writer,NOTICE OF PROPOSED PROCUREMENT\n\nFile Number:...
393,Industry Sustainment Business Case Analysis (S...,Industry Sustainment Business Case Analysis (S...
317,Invitation to Qualify to Artificial Intelligen...,This publication replaces the following AI Sou...
267,Request for Information for Environment and Cl...,ECCC is launching this RFI to engage with and ...
489,Request for Information Surveillance of Space ...,"Through this Request for Information (RFI), Pu..."


In [19]:
f = lambda x: '*Industrial optics' in x
df_optics = df_.loc[df.unspscDescription.apply(f), :]
df_optics

,title,tenderClosingDate,unspscDescription,contractingEntityName,tenderDescription
267,Request for Information for Environment and Cl...,2024-07-26T15:00:00,*Live animals\n*Domestic pet products\n*Animal...,Environment and Climate Change Canada,ECCC is launching this RFI to engage with and ...
489,Request for Information Surveillance of Space ...,2024-06-25T14:00:00,*Industrial optics\n*Lenses and prisms\n*Mirro...,Department of Public Works and Government Serv...,"Through this Request for Information (RFI), Pu..."
